In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scikitplot as plot
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, svm
from sklearn.model_selection import cross_val_score
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import datetime as dt
import math 
from sklearn import metrics
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import tree
import graphviz
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV as rs

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
#show data
print(df)
#unique number of values per column
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
# keep only numerical columns
df = df.select_dtypes(include =[np.number]) 

# remove the Outlier of skin thickness
max_skinthickness = df.SkinThickness.max()
df = df[df.SkinThickness!=max_skinthickness]

# replace zeros w mean values
def replace_zero(df, field, target):
    mean_by_target = df.loc[df[field] != 0, [field, target]].groupby(target).mean()
    df.loc[(df[field] == 0)&(df[target] == 0), field] = mean_by_target.iloc[0][0]
    df.loc[(df[field] == 0)&(df[target] == 1), field] = mean_by_target.iloc[1][0]

    # run the function
for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','Age','Pregnancies']:   
    replace_zero(df, col, 'Outcome')  


df.describe()

In [ ]:
features = df[["Glucose", "Insulin", "BMI",'BloodPressure']]
labels = df.Outcome

features.shape

In [ ]:
# Compute correlations
corr = df.corr()
# Exclude duplicate correlations by masking uper right values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# Add diverging colormap
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(250,10, as_cmap=True)
# Draw correlation plot, shrink legend
sns.heatmap(corr, mask=mask, cmap=cmap, 
        square=True,
        linewidths=.5, cbar_kws={"shrink": .5},annot=True, annot_kws={"size":12}, ax=ax)
plt.title("Diabetes Correlations")
plt.show ()

In [ ]:
sns.countplot(df['Outcome'])
plt.title("Diabetic Measurement 0 and 1")

# Scale/Features/ Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ss = StandardScaler()
scaler = MinMaxScaler()
features_train,features_test,labels_train,labels_test = train_test_split(features,labels,stratify=df.Outcome,test_size=0.2)

features_train = scaler.fit_transform(features_train)
features_test = scaler.fit_transform(features_test)
features_train= ss.fit_transform(features_train)
features_test = ss.fit_transform(features_test)

In [ ]:
#RANDOM FOREST
rf = RandomForestRegressor()
rf.fit(features_train,labels_train)
rf.score(features_test,labels_test)
#print(rf.feature_importances_)

In [ ]:
#DTClassifier
from sklearn.tree import DecisionTreeClassifier 
dtclf = DecisionTreeClassifier()
dtclf.fit(features_train,labels_train)
dtclf.score(features_test,labels_test)

In [ ]:
#fig = plt.figure(figsize = (10,10))
figsize = (4,4)
tree_graph = tree.export_graphviz(dtclf, out_file=None,filled = True)
graphviz.Source(tree_graph)

In [ ]:
tree_para = { 'max_leaf_nodes':[2,10,30],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150], 'criterion':['gini','entropy']}
dtclf = rs(DecisionTreeClassifier(), tree_para, cv=10)
dtclf.fit(features_train,labels_train)
dtclf.best_params_

In [ ]:
dtclf = DecisionTreeClassifier(max_leaf_nodes= 10,max_depth= 50,criterion= 'entropy')
dtclf.fit(features_train,labels_train)
dtclf.score(features_test,labels_test)

In [ ]:
#SVM
from sklearn import svm
clf = svm.SVC(kernel="linear")
clf.fit(features_train,labels_train)
clf.score(features_test,labels_test)

In [ ]:
param_grid = {'C': [0.1,1, 10], 'gamma': [1,0.1,0.01,0.001]}
               
svmclf = rs(clf, param_grid, cv=10)
svmclf.fit(features_train,labels_train)
print(svmclf.best_params_)
#print(svmclf.best_estimator_)

In [ ]:
svmclf = svm.SVC(C=10,gamma= 1,kernel="linear")
svmclf.fit(features_train,labels_train)
svmclf.score(features_test,labels_test)

In [ ]:
#Naive Bayes Classifier
from sklearn import naive_bayes
nbclf = naive_bayes.GaussianNB()
nbclf.fit(features_train,labels_train)
nbclf.score(features_test,labels_test)

In [ ]:
#Light GBM
import lightgbm as lgb

lgbclf = lgb.LGBMClassifier()
lgbclf.fit(features_train,labels_train)
lgbclf.score(features_test,labels_test)

In [ ]:
param_grid = {'n_estimators': [200, 400, 500],
    'max_depth': [15,20,25],
    'num_leaves': [25, 50, 75]}
               
lgbclf = rs(lgbclf, param_grid, cv=10)
lgbclf.fit(features_train,labels_train)
print(lgbclf.best_params_)
#print(lgbclf.best_estimator_)

In [ ]:
lgbclf = lgb.LGBMClassifier(num_leaves= 25, n_estimators= 400,max_depth= 15)
lgbclf.fit(features_train,labels_train)
print(lgbclf.score(features_test,labels_test))

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knnclf = KNeighborsClassifier(n_neighbors=2)
knnclf.fit(features_train,labels_train)
print(knnclf.score(features_test,labels_test))

In [ ]:
#In case of classifier like knn the parameter to be tuned is n_neighbors
param_grid = {'leaf_size' : [0,5,10,50], 'n_neighbors':np.arange(1,50)}

knnclf = KNeighborsClassifier()
knnclf= rs(knnclf,param_grid,cv=10)
knnclf.fit(features_test,labels_test)
print(knnclf.best_params_)
#print(knnclf.best_estimator_)
#print(knnclf.score(features_test,labels_test))

In [ ]:
knnclf = KNeighborsClassifier( n_neighbors=3,leaf_size=10, n_jobs=-1)
knnclf.fit(features_train,labels_train)

#print(cross_val_score(knnclf,features_test,labels_test,cv=10))
print(knnclf.score(features_test,labels_test))

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()
clf1.fit(features_train,labels_train)
clf1.score(features_test,labels_test)

In [ ]:
param_grid = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100]}
rs_search =rs(clf1, param_grid, cv=10)
rs_search.fit(features_train,labels_train)
print(rs_search.best_params_)
#print(rs_search.best_estimator_)

In [ ]:
clf1 = LogisticRegression(C= .1, n_jobs=-1, penalty='l2',
                   random_state=10)
clf1.fit(features_train,labels_train)
print(clf1.score(features_test,labels_test))
#print(cross_val_score(clf1,features_test,labels_test,cv=10))

In [ ]:
#XGBOOST
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()
xgb.fit(features_train,labels_train)
xgb.score(features_test,labels_test)

In [ ]:
param_grid = {
    'learning_rate': [.03, 0.05, .07],
              'max_depth': [5,6,7],
              'n_estimators':[200,300]}
#xgb = rs(xgb, param_grid, cv=10)
xgb = rs(xgb, param_grid, cv=10)
xgb.fit(features_train,labels_train)
print(xgb.best_params_)

In [ ]:
xgb = XGBRegressor(n_estimators= 200, learning_rate= .03,
              max_depth= 5)
xgb.fit(features_train,labels_train)
xgb.score(features_test,labels_test)

In [ ]:
algos = ["Support Vector Machine","Decision Tree","Logistic Regression","K Nearest Neighbor","Naive Bayes","Light GBM","XGBRegressor"]
clfs = [dtclf, svmclf, nbclf, lgbclf, knnclf, clf1, xgb]

result = []

for clff in clfs:
    clff.fit(features_train,labels_train)
    acc = clff.score(features_test,labels_test)
    result.append(acc)
result_df = pd.DataFrame(result,index=algos)
result_df.columns=["Tuned Parameters Accuracy"]
result_df.sort_values(by="Tuned Parameters Accuracy",ascending=False)

In [ ]:
#dtclf, svmclf, nbclf, lgbclf, knnclf, clf1, xgb


In [ ]:
#Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold =KFold(n_splits=10)

#algos = ["Support Vector Machine","Decision Tree","Logistic Regression","K Nearest Neighbor","Naive Bayes","Light GBM","XGBRegressor"]
#clfs = [dtclf, svmclf, nbclf, lgbclf, knnclf, clf1, xgb]

algos = ["Support Vector Machine","Decision Tree","Logistic Regression","K Nearest Neighbor","Naive Bayes","Light GBM"]
clfs = [dtclf, svmclf, nbclf, lgbclf, knnclf, clf1]

cv_results=[]
for classifiers in clfs:
    cv_score = cross_val_score(classifiers,features,labels,cv=kfold,scoring="accuracy")
    cv_results.append(cv_score.mean())
cv_mean = pd.DataFrame(cv_results,index=algos)
cv_mean.columns=["K-Fold10,CV Accuracy"]
cv_mean.sort_values(by="K-Fold10,CV Accuracy",ascending=False)

# Ensemble Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss
#voting_clf = VotingClassifier(estimators=[('SVC', svmclf), ('DTree', dtclf), ('LogReg', clf1), 
#                                          ('NaiveB', nbclf), ('KNN', knnclf), ('LGBM', lgbclf)], voting='hard')

voting_clf = VotingClassifier(estimators=[('SVC', svmclf),('KNN', knnclf),('NaiveB', nbclf)], voting='hard')
voting_clf.fit(features_train,labels_train)
preds = voting_clf.predict(features_test)

acc =  accuracy_score(labels_test, preds)
l_loss = log_loss(labels_test, preds)
f1 = f1_score(labels_test, preds)
cv= cross_val_score(voting_clf,features,labels,cv=kfold,scoring="accuracy")

#print("Accuracy is: " + str(acc))
print("Log Loss is: " + str(l_loss))
print("F1 Score is: " + str(f1))
print("Cross Val Score is: " + str(cv))


voting_clf = voting_clf.fit(features_train,labels_train)
pred = voting_clf.predict(features_test)
print('Accuracy on test:' , (labels_test == pred).mean())

In [ ]:
print(confusion_matrix(labels_test,pred))


# Stacking

In [ ]:
from vecstack import stacking
#models=dtclf, svmclf, nbclf, lgbclf, knnclf, clf1,xgb
models= svmclf,knnclf, lgbclf
S_train, S_test = stacking(models,                   
                           features_train, labels_train, features_test,   
                           regression=False, 
                           mode='oof_pred_bag',
                           needs_proba=False,
                           save_dir=None, 
                           metric=accuracy_score, 
                           n_folds=10, 
                           stratified=True,
                           shuffle=True,
                           random_state=0,    
                           verbose=2)

In [ ]:
def heldout_score(clf, X_test_cv, y_test_cv):
    score = np.zeros((n_estimators,), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test_cv)):
        score[i] = clf.loss_(y_test_cv, y_pred)
    return score

def cv_estimate(n_splits=10):
    cv = KFold(n_splits=n_splits)
    cv_clf = model
    val_scores = np.zeros((n_estimators,), dtype=np.float64)
    for train, test in cv.split(X_train_cv):
        cv_clf.fit(X_train_cv.iloc[train], y_train_cv[train])
        val_scores += heldout_score(cv_clf, X_train_cv.iloc[test], y_train_cv[test])
    val_scores /= n_splits
    return val_scores

def train_clf(lgbm, features_train, labels_train):
    return clf.fit(features_train, labels_train)

def train_predict(lgbm,features_train, labels_train, features_test, labels_test):
    train_clf(lgbm, features_train, labels_train)

In [ ]:
from sklearn.metrics import accuracy_score

params = {'n_estimators': [200, 400, 500],
    'max_depth': [15,20,25],
    'num_leaves': [25, 50, 75]}
lgbm = lgb.LGBMClassifier()

lgbm = rs(lgbm, params,cv=10)
lgbm.fit(features_train,labels_train)
print(lgbm.best_params_)

In [ ]:
lgbm = lgb.LGBMClassifier(num_leaves= 25, n_estimators= 400, max_depth= 25)
clf_ = lgbm.fit(features_train, labels_train)
y_pred = clf.predict(labels_test)

#train_predict(lgbm, features_train, labels_train, features_test, labels_test)
print('Accuracy is {}'.format(accuracy_score(features_test,y_pred )))

# Factors of All data, Diabetic and Non

In [ ]:
# Visualizing the distibution of the data for every feature
plt.figure(figsize=(10, 10))

for i, column in enumerate(df.columns, 1):
    plt.subplot(3, 3, i)
    df[df["Outcome"] == 0][column].hist(bins=35, color='blue', label='Have Diabetes = NO', alpha=0.6)
    df[df["Outcome"] == 1][column].hist(bins=35, color='red', label='Have Diabetes = YES', alpha=0.6)
    plt.legend()
    plt.xlabel(column)
  




In [ ]:
# Visualizing the distibution of the data for every feature
plt.figure(figsize=(10, 10))

for i, column in enumerate(features.columns, 1):
    plt.subplot(3, 3, i)
    df[df["Outcome"] == 0][column].hist(bins=35, color='blue', label='Have Diabetes = NO', alpha=0.6)
    df[df["Outcome"] == 1][column].hist(bins=35, color='red', label='Have Diabetes = YES', alpha=0.6)
    plt.legend()
    plt.xlabel(column)
  

In [ ]:
from itertools import combinations
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
names = ["Decision Tree","Logistic Regression","K Nearest Neighbor","Naive Bayes","Light GBM"]
clfs = [dtclf, nbclf, lgbclf, knnclf, clf1]
    
    ensemble = SuperLearner(scorer = accuracy_score, 
                            random_state = 0.5, 
                            folds = 10)
    ensemble.add(clf[0])
    ensemble.add_meta(lr)
    ensemble.fit(feature_train, labels_train)
    preds = ensemble.predict(features_test)
    accuracy = accuracy_score(preds, labels_test)
    
    if accuracy > best_combination[0]:
        best_combination[0] = accuracy
        best_combination[1] = clf[1]
    
    print("Accuracy score: {:.3f} {}").format(accuracy, clf[1])
    print("\nBest stacking model is {} with accuracy of: {:.3f}").format(best_combination[1], best_combination[0])

# Diabetics vs ALL Factors

In [ ]:
diabeticage = df['Outcome'] + df['Age']

In [ ]:
fig = plt.figure(figsize = (30,10))
sns.set_style("whitegrid")

sns.lineplot(x= diabeticage, y= df['Glucose'])
sns.lineplot(x= diabeticage, y= df['BMI'])
sns.lineplot(x= diabeticage, y= df['DiabetesPedigreeFunction'])
sns.lineplot(x= diabeticage, y= df['BloodPressure'])

plt.legend()
plt.title("Diabetics vs ALL Factors")
plt.xticks 




plt.show ()